In [6]:
import csv
import mysql.connector

def clean_item_weight(val):
    if val is None:
        return None
    val = val.strip()
    if val.lower() in ('', 'na', 'n/a', 'null', 'none', '--'):
        return None
    try:
        return float(val)
    except ValueError:
        return None

# Map CSV headers to MySQL columns
column_mapping = {
    'Item Fat Content': 'Item_Fat_Content',
    'Item Identifier': 'Item_Identifier',
    'Item Type': 'Item_Type',
    'Outlet Establishment Year': 'Outlet_Establishment_Year',
    'Outlet Identifier': 'Outlet_Identifier',
    'Outlet Location Type': 'Outlet_Location_Type',
    'Outlet Size': 'Outlet_Size',
    'Outlet Type': 'Outlet_Type',
    'Item Visibility': 'Item_Visibility',
    'Item Weight': 'Item_Weight',
    'Sales': 'Sales',
    'Rating': 'Rating'
}

# Connect to MySQL
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='sneha@#$2000',
    database='Blinkit_db'
)
cursor = connection.cursor()

rows_inserted = 0
rows_skipped = 0

# Open and read the CSV
with open("C:/Users/ADMIN/Downloads/Neha/Blinkit/BlinkIT_Grocery_Data.csv", 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)

    # Handle headers with comma as single string (bad CSV formatting)
    if len(header) == 1:
        header = header[0].split(',')

    print("CSV Header:", header)

    try:
        db_columns = [column_mapping[col.strip()] for col in header]
    except KeyError as e:
        print(f"❌ Column mismatch: {e}")
        exit()

    placeholders = ', '.join(['%s'] * len(db_columns))
    columns_str = ', '.join(db_columns)

    for row in reader:
        if len(row) == 1:
            row = row[0].split(',')

        cleaned_row = []
        for i, val in enumerate(row):
            val = val.strip()
            if header[i].strip() == 'Item Weight':
                cleaned_row.append(clean_item_weight(val))
            else:
                cleaned_row.append(val if val else None)

        try:
            sql = f"INSERT INTO blinkit ({columns_str}) VALUES ({placeholders})"
            cursor.execute(sql, cleaned_row)
            rows_inserted += 1
        except Exception as e:
            print(f"Error inserting row: {cleaned_row}")
            print(f"-> {e}")
            rows_skipped += 1

connection.commit()
cursor.close()
connection.close()

print(f"✅ Rows inserted: {rows_inserted}")
print(f"⚠️ Rows skipped: {rows_skipped}")


CSV Header: ['Item Fat Content', 'Item Identifier', 'Item Type', 'Outlet Establishment Year', 'Outlet Identifier', 'Outlet Location Type', 'Outlet Size', 'Outlet Type', 'Item Visibility', 'Item Weight', 'Sales', 'Rating']
✅ Rows inserted: 8523
⚠️ Rows skipped: 0
